<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/spark_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf /content/Spark-TTS && git clone https://github.com/weedge/Spark-TTS.git

In [ ]:
!cd /content/Spark-TTS/ && pip install -q -r requirements.txt

# download ckpt


In [6]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential


In [8]:
!huggingface-cli download SparkAudio/Spark-TTS-0.5B --quie --local-dir /content/Spark-TTS/pretrained_models/SparkTTS-0.5B


/content/Spark-TTS/pretrained_models/SparkTTS-0.5B


# Bicodec

In [23]:
!cd /content/Spark-TTS/ && PYTHONPATH=/content/Spark-TTS python sparktts/models/bicodec.py

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Missing tensor: mel_transformer.spectrogram.window
Missing tensor: mel_transformer.mel_scale.fb
BiCodec(
  (encoder): Encoder(
    (encoder): VocosBackbone(
      (embed): Conv1d(1024, 384, kernel_size=(7,), stride=(1,), padding=(3,))
      (norm): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (convnext): ModuleList(
        (0-11): 12 x ConvNeXtBlock(
          (dwconv): Conv1d(384, 384, kernel_size=(7,), stride=(1,), padding=(3,), groups=384)
          (norm): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (pwconv1): Linear(in_features=384, out_features=2048, bias=True)
          (act): GELU(approximate='none')
          (pwconv2): Linear(in_features=2048, out_features=384, bias=True)
        )
      )
      (final_layer_norm):

# tokenizer

In [18]:
!cd /content/Spark-TTS/ && PYTHONPATH=/content/Spark-TTS TOKENIZER_PATH=/content/Spark-TTS/pretrained_models/SparkTTS-0.5B/LLM python sparktts/utils/token_parser.py

[165137, 165027, 165032, 164053, 164965, 164992, 165022, 165035]
2025-03-12 09:15:43.732829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741770943.753609    5761 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741770943.758682    5761 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 09:15:43.776965: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
decode <|task_tts|><|age_0|><|gender_0|><|pitch_value_100|><

# inference

In [9]:
%cd /content/Spark-TTS/

/content/Spark-TTS


In [14]:
!PYTHONPATH=/content/Spark-TTS/ python cli/inference.py \
    --text "身临其境，换新体验。塑造开源语音合成新范式，让智能语音更自然。" \
    --device 0 \
    --save_dir "example/results" \
    --model_dir /content/Spark-TTS/pretrained_models/SparkTTS-0.5B \
    --prompt_text "吃燕窝就选燕之屋，本节目由26年专注高品质燕窝的燕之屋冠名播出。豆奶牛奶换着喝，营养更均衡，本节目由豆本豆豆奶特约播出。" \
    --prompt_speech_path "example/prompt_audio.wav"


2025-03-12 09:05:18.495380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741770318.534494    3177 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741770318.545084    3177 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 09:05:18.579951: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-12 09:05:22,980 - INFO - Using model from: /content/Spark-TTS/pretrained_models/SparkTTS-0.5B
2025-03-12 09:05:22,980

In [16]:
from IPython.display import Audio
Audio("example/results/20250312090342.wav",autoplay=True)

# streaming infer